In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt

In [ ]:
df = pd.read_csv("../input/demographics/demographics_blckgrp.csv")
print(df.shape)
df['geometry'] = df['geometry'].apply(wkt.loads)
to_drop = []
for col in df.columns:
    if ("error" in col or ".1" in col or "Not" in col): to_drop.append(col)
df = df.drop(to_drop, axis=1)
df.head(5)

In [ ]:
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
gdf.plot()

In [ ]:
df.iloc[3]

In [ ]:
cols_of_interest = ['White alone', 'Black or African American alone', 'American Indian and Alaska Native alone', 
                    'Asian alone','Native Hawaiian and Other Pacific Islander alone', 'Some other race alone',
                   'Hispanic or Latino:', 'Two or more races:']
df[cols_of_interest].head(5)

In [ ]:
import plotly.express as px
import geopandas as gpd
import shutil, os
shutil.rmtree("out")
os.mkdir("out")
# os.remove("stuff.zip")
def gen_and_save_map_plotly(work_type):
    work_str = work_type.replace(" ", "-").replace(":", "").lower()
    print(work_str)
    work_type_map = gdf[gdf[work_type].isnull() != True]
    fig = px.choropleth(work_type_map,
                       geojson=work_type_map.geometry,
                       locations=work_type_map.index,
                       color=work_type,
                       projection="mercator")
    fig.update_geos(fitbounds="locations", visible=False)
    fig.write_html("out/"+work_str+".div", include_plotlyjs=False, full_html=False)
    return True
list(map(gen_and_save_map_plotly, cols_of_interest))


In [ ]:
import os
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))
      
zipf = zipfile.ZipFile('files.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('out/', zipf)
zipf.close()